### Unemployment rate
we combine HBAI data with unemployment rate data from NOMIS to check if there is a correlation between unemployment and low income levels.


In [189]:
import pandas as pd

import plotly.express as px
import sys

    
def extract_and_fix_year(year_range):
    try:
        year_suffix = year_range.split('-')[1]
        full_year = "20" + year_suffix  if int(year_suffix) < 50 else "19" +year_suffix
        return int(full_year)  # Convert to integer
    except:
        return None


def read_HBAI_low_income(data_file):

    # Read the CSV file, skipping the first row (header row) and unnecessary footer rows
    df = pd.read_csv(data_file, skiprows=1, skipfooter=0, engine='python')

    # Drop empty columns and rows
    df = df.dropna(how='all', axis=1).dropna(how='all', axis=0)
    df['Financial Year'] = df['Financial Year'].str.extract(r'(\d{4}-\d{2})')
    df = df.dropna(subset=['Financial Year'])
    df.rename(columns={'Financial Year':'Year'},inplace=True)


    # convert year range to year, e.g. 1994-95 to 1995
    df['Year'] = df['Year'].apply(extract_and_fix_year)

    df.set_index('Year', inplace=True)

    # extract data columns.
    total_col_index = df.columns.str.contains('Total').argmax()

    #print(total_col_index)
    #not_low_income = df.iloc[:, :total_col_index+1].add_suffix('-not low income')

    # clean up the duplcated 1. 2. column names
    df.columns = df.columns.str.replace(r'\.\d+$', '', regex=True)
    
    df.columns =  [col.split('(')[0].strip() for col in df.columns]
    low_income = df.iloc[:, total_col_index + 1: total_col_index *2+2 ].add_suffix('-low income')

    print(low_income)
    total_columns = df.iloc[:, total_col_index *2 +2:].add_suffix('-total')

    # Combine the two groups of columns
    result = pd.concat([low_income, total_columns], axis=1)


    # Save or display the resulting DataFrame
    result.to_csv(data_file+"_processed.csv")  # Save to a new CSV file

    # convert to numbers
    result = result.apply(pd.to_numeric, errors='coerce')
    print(result.head(10))
    return result, low_income.columns


In [190]:

BHC, regions=read_HBAI_low_income("./HBAI-BHC-Region.csv")


     Northern Ireland-low income Scotland-low income Wales-low income  \
Year                                                                    
1995                          ..             1041639           642807   
1996                          ..              977205           651553   
1997                          ..             1091777           600766   
1998                          ..             1012349           701328   
1999                          ..              983394           631297   
2000                          ..             1040313           611574   
2001                          ..             1027484           626782   
2002                          ..              961666           622986   
2003                      318527              996094           587911   
2004                      337402              912433           633619   
2005                      344460              855978           559739   
2006                      339769              87348

In [209]:
region_list=list(regions.str.replace('-low income',''))
if 'Total' in region_list:
    region_list.remove('Total')
region_list

['Northern Ireland',
 'Scotland',
 'Wales',
 'North East',
 'North West',
 'Yorkshire and The Humber',
 'East Midlands',
 'West Midlands',
 'East',
 'London',
 'South East',
 'South West']

In [192]:
def read_employment_csv(data_file):

    # Read the CSV file, skipping the first row (header row) and unnecessary footer rows
    df = pd.read_csv(data_file, skiprows=0, skipfooter=0, engine='python')
    #print(df)
    # Drop empty columns and rows
    #df = df.dropna(how='all', axis=1).dropna(how='all', axis=0)
    #print(df)
    # Extract the year column and clean it
    df['Year'] = df['Date'].str.extract(r'(\d{4})').astype(int)
    #print(df['Year'])
    print(df.columns)
    # Set the year as the index
    df.set_index('Year', inplace=True)
    df.drop(columns=['Date'], inplace=True)
    
    # convert to numbers
    df = df.apply(pd.to_numeric, errors='coerce')
    print(df.head(10))
    df.to_csv(f'{data_file}_processed.csv')
    
    return df


In [194]:
unemp_df = read_employment_csv("./NOMIS-unemployment-region.csv")


Index(['Date', 'East', 'East Midlands', 'London', 'North East', 'North West',
       'Northern Ireland', 'Scotland', 'South East', 'South West', 'Wales',
       'West Midlands', 'Yorkshire and The Humber', 'Year'],
      dtype='object')
      East  East Midlands  London  North East  North West  Northern Ireland  \
Year                                                                          
2004   3.8            4.4     7.1         5.7         4.9               4.6   
2005   4.6            5.0     8.0         6.6         5.2               4.8   
2006   4.3            5.0     7.0         6.4         5.7               4.2   
2007   4.3            5.3     6.9         6.8         5.8               3.8   
2008   6.0            7.2     8.5         9.1         8.2               6.3   
2009   6.6            7.4     9.1         9.6         8.2               6.8   
2010   6.6            7.6     9.5        10.3         8.1               7.0   
2011   6.9            8.2     9.2        10.8       

In [ ]:
# calculate low income percent (per region)
low_income_pct_df = pd.DataFrame()
for region in region_list:
    low_income_pct_df[f'{region}'] = round((BHC[f'{region}-low income']/BHC[f'{region}-total'])*100,2)
print(low_income_pct_df)

low_income_pct_df.to_csv("BHC.lowincome.pct.csv")



      Northern Ireland  Scotland  Wales  North East  North West  \
Year                                                              
1995               NaN     20.77  22.62       22.71       21.45   
1996               NaN     19.49  22.93       25.04       18.87   
1997               NaN     21.83  21.12       24.03       21.71   
1998               NaN     20.29  24.63       24.09       22.14   
1999               NaN     19.75  22.14       26.24       23.87   
2000               NaN     20.92  21.44       26.10       21.14   
2001               NaN     20.71  21.93       24.60       20.12   
2002               NaN     19.39  21.78       21.33       21.01   
2003             19.06     20.00  20.49       23.41       19.13   
2004             20.10     18.31  21.97       20.76       19.06   
2005             20.40     17.13  19.28       21.22       18.07   
2006             19.96     17.40  20.44       19.56       20.08   
2007             19.21     16.58  20.70       21.00       20.7

#### Find out the correlation between the two over the overlapping time period.


In [195]:
unemp_df=unemp_df.reset_index()
low_income_pct_df=low_income_pct_df.reset_index()


In [197]:
low_income_pct_df=low_income_pct_df.dropna()


#unemp_df_correlation = unemp_df[unemp_df['Year'].isin(low_income_pct_df.Year)].drop('Year', axis=1)
#low_income_pct_df_correlation = low_income_pct_df[low_income_pct_df['Year'].isin(unemp_df.Year)].drop('Year', axis=1)
unemp_df_correlation = unemp_df[unemp_df['Year'].isin(low_income_pct_df.Year)]
low_income_pct_df_correlation = low_income_pct_df[low_income_pct_df['Year'].isin(unemp_df.Year)]

print(unemp_df_correlation)
print(low_income_pct_df_correlation)

unemp_df_correlation.to_csv("unemployment.csv")
low_income_pct_df_correlation.to_csv("lowincome.csv")

    Year  East  East Midlands  London  North East  North West  \
0   2004   3.8            4.4     7.1         5.7         4.9   
1   2005   4.6            5.0     8.0         6.6         5.2   
2   2006   4.3            5.0     7.0         6.4         5.7   
3   2007   4.3            5.3     6.9         6.8         5.8   
4   2008   6.0            7.2     8.5         9.1         8.2   
5   2009   6.6            7.4     9.1         9.6         8.2   
6   2010   6.6            7.6     9.5        10.3         8.1   
7   2011   6.9            8.2     9.2        10.8         8.7   
8   2012   6.3            7.7     8.9        10.2         7.9   
9   2013   5.3            5.9     7.2         8.8         7.4   
10  2014   4.2            5.0     6.4         7.6         5.9   
11  2015   3.8            4.2     6.0         6.8         5.1   
12  2016   3.5            4.2     5.4         6.4         4.7   
13  2017   3.8            4.3     5.1         5.2         4.1   
14  2018   3.0           

We check the correlation for each region.

In [208]:
from scipy.stats import pearsonr

for region in region_list:
    if region=='Total': continue
    corr, p_value = pearsonr(list(unemp_df_correlation[f'{region}']), list(low_income_pct_df_correlation[f'{region}']))
    print(f'correlation: {region} {corr} {p_value}')


correlation: Northern Ireland 0.6958572470665688 0.000937409479517699
correlation: Scotland -0.5496484839200828 0.014774855272064145
correlation: Wales -0.0007055899326945808 0.9977126462165471
correlation: North East 0.006936815861455267 0.977515179249301
correlation: North West -0.33061777467509734 0.1668058167969946
correlation: Yorkshire and The Humber 0.002211639925542949 0.9928304711378592
correlation: East Midlands 0.002952083901549697 0.9904302467208203
correlation: West Midlands -0.04613997812553521 0.8512178203437736
correlation: East 0.017811000753589 0.9423066874697116
correlation: London 0.4266015004821272 0.06853748119997574
correlation: South East -0.2997820389732645 0.2124228337116656
correlation: South West 0.1642323285369292 0.5016745994201735


The correlation values do not look strong.  However this might not be very accurate due to the small sample size.  So we plot a scatter chart and observe how the two moves with each other for all regions.

In [223]:

# melt for plotting
#low_income_pct_df=low_income_pct_df.reset_index()
hbai_plot_data = low_income_pct_df.melt(
        id_vars=['Year'],
        value_vars=region_list,
        var_name='Regions',
        value_name='Low income pct'
    )

unemp_df_plot_data = unemp_df.melt(
        id_vars=['Year'],
        value_vars=region_list,
        var_name='Regions',
        value_name='unemp pct'
    )

hbai_plot_data

,Year,Regions,Low income pct
0,2003,Northern Ireland,19.06
1,2004,Northern Ireland,20.10
2,2005,Northern Ireland,20.40
3,2006,Northern Ireland,19.96
4,2007,Northern Ireland,19.21
...,...,...,...
235,2018,South West,14.71
236,2019,South West,14.14
237,2020,South West,14.33
238,2022,South West,15.91


In [ ]:


combined = unemp_df_plot_data.merge(hbai_plot_data, on=['Year','Regions'])
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
print(combined)
# Scatter plot with Plotly Express
fig = px.scatter(combined, x='unemp pct', y='Low income pct', color='Regions', 
                 #text='Regions', 
                 hover_data=['Year'], title='Unemployment Rate vs. HBAI Percentage (2004-2023)')


# Add trend line
m, b = np.polyfit(combined['unemp pct'], combined['Low income pct'], 1)
trend_x = np.array([min(combined['unemp pct']), max(combined['unemp pct'])])
trend_y = m * trend_x + b
fig.add_trace(go.Scatter(x=trend_x, y=trend_y, mode='lines', name='Trend Line', line=dict(color='red')))

# Update layout
fig.update_traces(textposition='top center')
fig.update_layout(xaxis_title='Unemployment Rate (%)', yaxis_title='HBAI Percentage (%)', 
                  showlegend=True, height=600, width=800)

# Show plot
fig.show()

     Year           Regions  unemp pct  Low income pct
0    2004  Northern Ireland        4.6           20.10
1    2005  Northern Ireland        4.8           20.40
2    2006  Northern Ireland        4.2           19.96
3    2007  Northern Ireland        3.8           19.21
4    2008  Northern Ireland        6.3           19.54
..    ...               ...        ...             ...
223  2018        South West        3.3           14.71
224  2019        South West        3.5           14.14
225  2020        South West        3.7           14.33
226  2022        South West        2.5           15.91
227  2023        South West        2.6           13.87

[228 rows x 4 columns]


Visually, there appears to be a positive correlation, as the trend line slopes upward. However, the scatter suggests that the relationship isn’t perfectly linear, and there’s some variability.  So unemployment alone doesn’t fully explain low income.
